<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/Signature_Verification_Nasnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [34]:
# Importing the Keras libraries and packages
import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3
from keras.applications.nasnet import NASNetMobile
from keras.callbacks.callbacks import EarlyStopping
from keras.layers import Dropout
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 224, 224
print("All Libraries are loaded")

All Libraries are loaded


In [35]:
# Loading pretrained Network
nasnet_model=NASNetMobile(input_shape=(img_height,img_width,3), include_top=False, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
print("pretrained Network is loaded")

19996672/19993432 [==============================] - 0s 0us/step
pretrained Network is loaded


In [36]:
nasnet_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [38]:
# Freeze the layers
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in nasnet_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [40]:
model = Sequential()
model.add(nasnet_model)
#add fully connected layer:
model.add(Flatten())
model.add(Dense(128, activation='relu', name='hidden_layer1'))
model.add(Dense(128, activation='relu', name='hidden_layer2'))
model.add(Dense(64, activation='relu', name='classification_layer'))
keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
output_layer=model.add(Dense(1, activation='sigmoid', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [41]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset1/training_set'
val_data_dir ='/content/drive/My Drive/Dataset1/testing_set'
model_weights_file = 'NASNet-mobile-no-top.h5'
nb_epochs = 35
print("Input parameters are assigned")

Input parameters are assigned


In [42]:
# Compliling Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [43]:
# Mounting google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Accesig dataset and put into generator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=False,class_mode='binary')


Found 2112 images belonging to 2 classes.
Found 544 images belonging to 2 classes.


In [46]:
# Training of Model
filepath="/content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_weights.best.hdf5"
callbacks = [ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

history = model.fit_generator( train_generator, callbacks = callbacks, nb_epoch=nb_epochs, validation_data=validation_generator)

print('Training Completed!')

Epoch 1/35
66/66 [==============================] - 346s 5s/step - loss: 3.0249 - accuracy: 0.4957 - val_loss: 3.1569 - val_accuracy: 0.4926

Epoch 00001: val_accuracy improved from -inf to 0.49265, saving model to /content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_weights.best.hdf5
Epoch 2/35
66/66 [==============================] - 334s 5s/step - loss: 1.5148 - accuracy: 0.4967 - val_loss: 1.4234 - val_accuracy: 0.5202

Epoch 00002: val_accuracy improved from 0.49265 to 0.52022, saving model to /content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_weights.best.hdf5
Epoch 3/35
66/66 [==============================] - 332s 5s/step - loss: 0.9242 - accuracy: 0.4905 - val_loss: 0.6547 - val_accuracy: 0.5588

Epoch 00003: val_accuracy improved from 0.52022 to 0.55882, saving model to /content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_weights.best.hdf5
Epoch 4/35
66/66 [==========

In [49]:
# save model and architecture to single file
model.save("/content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_model.h5")
model.summary()

print("Saved model to disk")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 7, 7, 1056)        4269716   
_________________________________________________________________
flatten_1 (Flatten)          (None, 51744)             0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 128)               6623360   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 128)               16512     
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 10,917,909
Trainable params: 6,648,193
Non-trainable params: 4,269,716
___________________________________

In [51]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model("/content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_model.h5")
print("Model is Loaded")

Model is Loaded


In [52]:
intermediate_output = model.predict(train_generator)
X_train = np.array(intermediate_output)
print(X_train)
print(X_train.shape)

[[0.51991695]
 [0.51991695]
 [0.51991695]
 ...
 [0.51991695]
 [0.51991695]
 [0.51991695]]
(2112, 1)


In [53]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [54]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [56]:
# Saving intermediate model
intermediate_layer_model.save("/content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/intermediate_model_nasnet_model.h5")
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet_input (InputLayer)    (None, 224, 224, 3)       0         
_________________________________________________________________
NASNet (Model)               (None, 7, 7, 1056)        4269716   
_________________________________________________________________
flatten_1 (Flatten)          (None, 51744)             0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 128)               6623360   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 128)               16512     
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 10,917,844
Trainable params: 6,648,128
Non-trainable params: 4,269,716
________________________________________

In [57]:
intermediate_output = intermediate_layer_model.predict(train_generator)
X_train = np.array(intermediate_output)
print(X_train)
print(X_train.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2112, 64)


In [58]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model("/content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/intermediate_model_nasnet_model.h5")
print("Intermediate model is loaded")

Intermediate model is loaded


In [12]:
intermediate_output = model.predict(train_generator)
X_train = np.array(intermediate_output)
print(X_train)
print(X_train.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2112, 64)


In [13]:
intermediate_output_test = model.predict(validation_generator)
X_test = np.array(intermediate_output_test)
print(X_test)
print(X_test.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(544, 64)


In [59]:
# Training Label feature identification(y_train)
 import numpy as np
 batch_size=32
 sample_count=2112
 features = np.zeros(shape=(2112, 64))  # Must be equal to the output of the convolutional base
 labels = np.zeros(shape=(2112))
 i = 0
 for inputs_batch, labels_batch in train_generator:
   features_batch = model.predict(inputs_batch)
   features[i * batch_size: (i + 1) * batch_size] = features_batch
   labels[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels.shape)

(2112,)


In [60]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=np.expand_dims(labels, axis=1)
print(labels_train.shape)
print(labels_train)

(2112, 64)
(2112, 1)
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [61]:
# identification of test labels
 import numpy as np
 batch_size=32
 sample_count=544
 features_test = np.zeros(shape=(544, 64))  # Must be equal to the output of the convolutional base
 labels_test = np.zeros(shape=(544))
 i = 0
 for inputs_batch, labels_batch in validation_generator:
   features_batch = model.predict(inputs_batch)
   features_test[i * batch_size: (i + 1) * batch_size] = features_batch
   labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
   i += 1
   if i*batch_size  >= sample_count:
     break
 print(labels_test.shape)
 print(features_test.shape)
 print(labels_test)

(544,)
(544, 64)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

In [62]:
# #identification of testing Labels
print(features_test.shape)
labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)

(544, 64)
(544, 1)


In [64]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train,labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[  1 267]
 [  1 275]]
0.5073529411764706


In [65]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[  2 266]
 [  1 275]]
0.5091911764705882


In [66]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))


[[  3 265]
 [  6 270]]
0.5018382352941176


In [67]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[  5 263]
 [  7 269]]
0.5036764705882353


In [68]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[261   7]
 [250  26]]
0.5275735294117647


In [69]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[263   5]
 [250  26]]
0.53125


In [70]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, labels_train.ravel())

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print(cm)
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_test, y_pred))

[[  1 267]
 [  0 276]]
0.5091911764705882


In [72]:
from keras.models import load_model

model = load_model("/content/drive/My Drive/Colab Notebooks/Signature Verification Nasnet Model/model_nasnet_model.h5")
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
17/17 [==============================] - 72s 4s/step
test loss, test acc: [0.6489264369010925, 0.5055146813392639]
